In [ ]:
#steps to run 
#1- create astra db acoount and make one vector data base 
!pip install -q cassio datasets langchain tiktoken PyPDF2 sentence-transformers langchain-community

In [44]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
import cassio
from PyPDF2 import PdfReader
import re

In [45]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:XYQUCiXvsiPjptNAElONyhCX:e5f8f21a639dfb832b7bd829f771fa095818b00c389b3e82253c2d4277521078" ##enter astra db token
ASTRA_DB_ID = "8cc44c17-75b1-484d-be19-e3ce261956fd" #enter astra db id of same token 
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [46]:
class LocalEmbedding:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, text):
        return self.model.encode(text).tolist()

    def embed_documents(self, texts):
        return [self.model.encode(text).tolist() for text in texts]


In [47]:
local_embedding = LocalEmbedding()

In [48]:
pdfreader = PdfReader('pdfs/Amaar raja Earnings Summary.pdf')
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content + "\n"

In [49]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

In [50]:
astra_vector_store = Cassandra(
    embedding=local_embedding,  
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)


In [59]:
astra_vector_store.add_texts(texts)

['c486712f0798451f94eb7885e0187162',
 '659b389b4e8d4ae2a0b7ffe71978b788',
 'e26043df2c124102b582c3c20dbe0846',
 '86fda7d8e6204eeebb2cc890338fea08',
 'aa46fadba1824e8697264619810fa61e',
 'cd140dca22fd4fa0b585734e3a3108c2',
 'b9b8f8ca48394c418cdf972721e1c1f7',
 'acb45c707fed493b9e75f9338ca51024',
 '209c1e89133b40068825571f9cda5060',
 'ab14b74150984034a95ea67c39f95b2c',
 'f091224a216847eeab1d1031064a60b3',
 '6bba13e84e84418080151d85df36defa',
 'a554dc178f224b70bfdab35b2461aa25',
 'c2c57c0887644e5e8b2ee0b5952665f4',
 '48fe1e4075f3434d8498e50684e7e549',
 '1ab8e2fba64a471db96fdd61fcc55288',
 '7a69eb11608b423f95c2758ea30f6ed7',
 '358f657b2b3b4178b827b642aeb9a6a9',
 'f46423d1a1bb4681badae336e80fa12c',
 '870a21a74ecc4bafadf14eada2d16e23',
 '9c116aa871524f2d9d5b1f3c7cbbee6d',
 'acbb714eb7344b66b62177a5321fd854',
 'de56a3c094d64af79bed4e185dedc35d',
 '736a92bf32f14f25855ab5490ccbf9b8',
 '930c169282064ffa9bbd25ae5694fcbb',
 'd29c9d0abffd402a9837fa4bfbe20ca4',
 'dd0c1fca092a4caeb2fb396ac41ca6cf',
 

In [52]:
patterns = {
    "Company Name": r"(?i)(?:Company Name|Name of the Company|Corporate Name|Yours faithfully)[:\s]+([A-Za-z&.,\s-]+)",
    "Report Date": r"(?i)(?:Report Date|Dated|As on)[:\s]+([\dA-Za-z,\s]+)",
    "Profit Before Tax": r"(?i)Profit\s*Before\s*Tax\s*[\(]?.*?[\)]?[:\s]+([\d,.]+)",
    "Revenue from Operations": r"(?i)Revenue\s*from\s*Operations[:\s]+([\d,.]+)",
    "Total Income": r"(?i)Total\s*Income[:\s]+([\d,.]+)",
    "Total Expenses": r"(?i)Total\s*Expenses[:\s]+([\d,.]+)",
    "Net Profit After Tax": r"(?i)Net\s*Profit\s*After\s*Tax[:\s]+([\d,.]+)"
}

In [53]:
extracted_data = {key: "Not Found" for key in patterns.keys()}

In [54]:
for key, pattern in patterns.items():
    match = re.search(pattern, raw_text, re.MULTILINE)
    if match:
        extracted_data[key] = match.group(1).strip()

In [55]:
lines = raw_text.split("\n")
for line in lines:
    if extracted_data["Company Name"] == "Not Found" and re.search(r"(?i)(Company Name|Corporate Name)", line):
        extracted_data["Company Name"] = line.split(":")[-1].strip()
    if extracted_data["Report Date"] == "Not Found" and re.search(r"(?i)(Report Date|Dated|As on)", line):
        extracted_data["Report Date"] = line.split(":")[-1].strip()

In [56]:
if "meeting" in extracted_data["Company Name"].lower():
    extracted_data["Company Name"] = "Not Found"

In [57]:
print("\n===== Extracted Financial Report =====\n")
print(f"Company Name: {extracted_data['Company Name']}")
print(f"Report Date: {extracted_data['Report Date']}")
print(f" Profit Before Tax: ₹{extracted_data['Profit Before Tax']} crores")
print(f" Revenue from Operations: ₹{extracted_data['Revenue from Operations']} crores")
print(f" Total Income: ₹{extracted_data['Total Income']} crores")
print(f" Total Expenses: ₹{extracted_data['Total Expenses']} crores")
print(f" Net Profit After Tax: ₹{extracted_data['Net Profit After Tax']} crores")
print("\n=======================================\n")



===== Extracted Financial Report =====

Company Name: For Amara Raja Energy & Mobility Limited
Report Date: September 26, 2024, we wish to inform you that the Board of 
Directors at their meeting held today i
 Profit Before Tax: ₹323.97 crores
 Revenue from Operations: ₹3,250.73 crores
 Total Income: ₹3,154.30 crores
 Total Expenses: ₹2,830.33 crores
 Net Profit After Tax: ₹226.32 crores




In [58]:
import json

financial_report = {
    "Company Name": extracted_data['Company Name'],
    "Report Date": extracted_data['Report Date'],
    "Profit Before Tax": f"₹{extracted_data['Profit Before Tax']} crores",
    "Revenue from Operations": f"₹{extracted_data['Revenue from Operations']} crores",
    "Total Income": f"₹{extracted_data['Total Income']} crores",
    "Total Expenses": f"₹{extracted_data['Total Expenses']} crores",
    "Net Profit After Tax": f"₹{extracted_data['Net Profit After Tax']} crores"
}

with open("Details.json", "w", encoding="utf-8") as json_file:
    json.dump(financial_report, json_file, indent=4)
print("Saved")

Saved
